In [10]:
# import dependencies
import pandas as pd
import numpy as np
import json
import os
import requests
import io
from sklearn.model_selection import train_test_split, KFold
import tqdm
import h5py

In [ ]:
base_path_wsi = '/your/wsi/path'

In [2]:
# helpers function

def request_file_info(data_type):
    fields = [
        'file_name',
        'cases.submitter_id',
        'cases.samples.sample_type',
        'cases.project.project_id',
        'cases.project.primary_site',
        ]

    fields = ','.join(fields)

    files_endpt = 'https://api.gdc.cancer.gov/files'

    filters = {
        'op': 'and',
        'content':[
            {
            'op': 'in',
            'content':{
                'field': 'files.experimental_strategy',
                'value': [data_type]
                }
            }
        ]
    }

    params = {
        'filters': filters,
        'fields': fields,
        'format': 'TSV',
        'size': '2000000'
        }

    response = requests.post(files_endpt, headers = {'Content-Type': 'application/json'}, json = params)

    return pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')

In [3]:
def make_patient_file_map(df, base_dir):
    """
    Adapted from Vale-Silva's preprocessing.
    """
    
    d = {}
    for _, row in df.iterrows():
        patient = row['cases.0.submitter_id']
        if os.path.exists(os.path.join(base_dir, row.id, row.file_name)):
            if patient in d:
                if not isinstance(d[patient], tuple):
                    d[patient] = (
                        d[patient],
                        os.path.join(base_dir, row.id, row.file_name))
                else:
                    d[patient] += os.path.join(base_dir, row.id, row.file_name),
            else:
                d[patient] = os.path.join(base_dir, row.id, row.file_name)

    return d

In [4]:
def get_n_patches(x):
    with h5py.File(x, "r") as f:
        n_patch = f["coords"].shape[1]

    return n_patch


def select_primary(df_group):
    #prioritize the first "Primary tumor" if available
    primary_tumors = df_group[df_group['cases.0.samples.0.sample_type'] == "Primary Tumor"]
    if not primary_tumors.empty:
        return primary_tumors.iloc[0]  # Take the first occurrence of "Primary tumor"
    else:
        return df_group.iloc[0]  # Otherwise, take the first row

# 2. Clinical Data <a name="p2"></a> 
Everything related to the preparation of the clinical dataframe is listed below.

In [5]:
clinical = pd.read_csv('files/clinical.tsv', sep='\t')

In [6]:
clinical.replace("'--", np.nan, inplace=True)
print((clinical.isna().sum()/clinical.shape[0]).sort_values(ascending=True))

case_id                           0.000000
case_submitter_id                 0.000000
project_id                        0.000000
ethnicity                         0.002144
gender                            0.002144
                                    ...   
irs_group                         1.000000
international_prognostic_index    1.000000
inss_stage                        1.000000
uicc_clinical_stage               1.000000
mitotic_count                     1.000000
Length: 197, dtype: float64


/tmp/ipykernel_3256672/413249852.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clinical.replace("'--", np.nan, inplace=True)


In [7]:
clinical.drop_duplicates(subset=['case_submitter_id'], inplace=True)
clinical.rename(columns={"case_submitter_id": "submitter_id"}, inplace=True)
clinical.drop(columns=['case_id'], inplace=True)

In [8]:
clinical_tcga = clinical[clinical.project_id.str.startswith('TCGA')]
clinical_cptac = clinical[clinical.project_id.str.startswith('CPTAC')]

In [9]:
clinical_tcga

,submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_birth,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_dose_units,treatment_duration,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_outcome_duration,treatment_type
0,TCGA-DD-AAVP,TCGA-LIHC,48,false,NaN,NaN,NaN,NaN,-17833,NaN,...,NaN,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,NaN,"Radiation Therapy, NOS"
19,TCGA-KK-A7B2,TCGA-PRAD,68,false,Not Reported,NaN,NaN,United States,-24845,NaN,...,NaN,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,NaN,"Radiation Therapy, NOS"
22,TCGA-DC-6158,TCGA-READ,70,false,NaN,NaN,NaN,United States,-25842,334,...,NaN,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,NaN,"Radiation Therapy, NOS"
24,TCGA-DD-A4NP,TCGA-LIHC,32,false,NaN,NaN,NaN,United States,-11838,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,"Ablation or Embolization, NOS"
42,TCGA-HQ-A5ND,TCGA-BLCA,78,false,NaN,NaN,NaN,Canada,-28555,274,...,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,"Pharmaceutical Therapy, NOS"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59221,TCGA-BP-4790,TCGA-KIRC,76,false,NaN,NaN,NaN,NaN,-28076,1111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59222,TCGA-N9-A4Q4,TCGA-UCS,61,false,NaN,NaN,NaN,United States,-22575,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,"Pharmaceutical Therapy, NOS"
59228,TCGA-RY-A847,TCGA-LGG,45,false,NaN,NaN,NaN,United States,-16732,NaN,...,NaN,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,NaN,"Pharmaceutical Therapy, NOS"
59231,TCGA-AB-2881,TCGA-LAML,48,false,NaN,NaN,NaN,NaN,-17808,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,"Pharmaceutical Therapy, NOS"


In [10]:
keep_cols = ['submitter_id', 'project_id', 'tumor_stage', 'age_at_diagnosis', 'prior_treatment', 'prior_malignancy',
             'synchronous_malignancy', 'gender', 'race', 'ethnicity']

columns_to_drop = [col for col in clinical_tcga.columns if col not in keep_cols]
clinical_tcga = clinical_tcga.drop(columns=columns_to_drop)

Take information from multisurv

In [11]:
multisurv_clinical = pd.read_csv('files/multisurv_clinical.tsv', sep='\t')[['submitter_id', 'tumor_stage', 'treatments_pharmaceutical_treatment_or_therapy', 'treatments_radiation_treatment_or_therapy']] 

In [12]:
clinical_tcga = clinical_tcga.merge(multisurv_clinical, how='left', on='submitter_id')

## Supplementary information

In [17]:
%%capture
supplementary = pd.read_csv('files/supplementary_data.tsv', sep='\t')

In [22]:
mapper_clin = {
    'patient': 'submitter_id', 
    'IDH.status': 'IDH', 
    'MGMT.promoter.status': 'MGMT',
    'X1p.19q.codeletion': 'X1p19q',
    'Chr.19.20.co.gain': '19.20.gain',
    'Chr.7.gain.Chr.10.loss': '7g10l',
    'TERT.expression.status': 'TERT',
    'ATRX.status': 'ATRX',
    'TP53 mutation': 'TP53'
}

In [23]:
supplementary = supplementary.drop(columns=['IDH', 'Mutations', 'TERT']).rename(columns=mapper_clin)[list(mapper_clin.values())]
supplementary = supplementary[~supplementary.duplicated(subset=['submitter_id'], keep='first')].dropna(subset='submitter_id')

In [24]:
supplementary['subtype'] = supplementary.apply(
    lambda row: (
        'glioblastoma' if row['IDH'] == 'WT' else
        ('oligodendrogliome' if row['X1p19q'] == 'codel' else
         'astrocytoma' if row['X1p19q'] == 'non-codel' else np.nan)
        if pd.notna(row['IDH']) else np.nan
    ),
    axis=1
)

In [25]:
clinical_tcga = clinical_tcga.merge(supplementary, how='left', on='submitter_id')

In [26]:
race_subset = clinical_tcga['race'].isnull()
ethnicity_subset = ~clinical_tcga['ethnicity'].isnull()
subset = race_subset & ethnicity_subset
clinical_tcga.loc[subset, 'race'] = clinical_tcga.loc[subset, 'ethnicity']
race_subset = (clinical_tcga['race'] == 'white')
ethnicity_subset = (~clinical_tcga['ethnicity'].isnull() &
                    (clinical_tcga['ethnicity'] == 'hispanic or latino'))
subset = race_subset & ethnicity_subset
clinical_tcga.loc[subset, 'race'] = clinical_tcga.loc[subset, 'ethnicity']

In [27]:
clinical_tcga = clinical_tcga.drop('ethnicity', axis=1)

## Get DSS labels according to SurvPath and MOTCat for TCGA-GBMLGG

For this part you will need to clone [SurvPath](https://github.com/mahmoodlab/SurvPath/tree/main), and [MOTCat](https://github.com/Innse/MOTCat).

# From MOTCAT:

In [24]:
motcat_columns = ['case_id', 'survival_months', 'censorship', 'age', 'site', 'oncotree_code']
labels_columns = ['case_id', 'survival_months', 'censorship', 'age', 'site', 'oncotree_code', 'survival_months_dss', 'censorship_dss']

def preprocess_df(df, columns):
    df = df[columns]
    df.rename(columns={'case_id': 'submitter_id'}, inplace=True)
    df.drop_duplicates(subset=['submitter_id'], inplace=True)
    return df

In [25]:
%%capture
gbmlgg = preprocess_df(pd.read_csv('motcat_metadata/tcga_gbmlgg.csv'), motcat_columns)
luad = preprocess_df(pd.read_csv('motcat_metadata/tcga_luad.csv'), motcat_columns)

In [26]:
%%capture
brca = preprocess_df(pd.read_csv('survpath_metadata/tcga_brca.csv'), labels_columns)
coadread = preprocess_df(pd.read_csv('survpath_metadata/tcga_coadread.csv'), labels_columns)

In [49]:
combined = pd.concat([gbmlgg, luad, brca, coadread])

In [32]:
clinical_tcga = clinical_tcga.merge(combined, how='left', on='submitter_id')

In [33]:
# make response
treatment_mask = (
    (clinical_tcga['treatments_pharmaceutical_treatment_or_therapy'] == 'yes') &
    (clinical_tcga['treatments_radiation_treatment_or_therapy'] == 'yes')
)

# Step 2: Add mask for censorship == 0
eligible_mask = treatment_mask & (clinical_tcga['censorship'] == 0)

# Step 3: Apply condition on survival
clinical_tcga['response'] = np.where(
    eligible_mask,
    np.where(clinical_tcga['survival_months'] > 15, True, False),
    np.nan
)

# 3. Whole Slide Images <a name="p3"></a>
UNI-v2

In [34]:
wsi_mapping = {}
for path in os.listdir(base_path_wsi):
    print(path)
    slides = os.listdir(os.path.join(base_path_wsi, path))
    for slide in tqdm.tqdm(slides):base_path_wsi
        patient_id = slide[:12]
        if patient_id in list(wsi_mapping.keys()):
            continue

        patient_slides = [os.path.join(base_path_wsi, path, slide) for slide in slides if slide.startswith(patient_id)]
        n_patches = [get_n_patches(p) for p in patient_slides]
        sorted_list = [x for x, _ in sorted(zip(patient_slides, n_patches), key=lambda pair: pair[1], reverse=True)]
        sorted_list = ['/'.join(p.split('/')[3:]) for p in sorted_list]
        wsi_mapping[patient_id] = sorted_list

TCGA-KICH


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:05<00:00, 20.07it/s]


TCGA-KIRC


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 519/519 [00:21<00:00, 23.65it/s]


TCGA-KIRP


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 158/158 [00:05<00:00, 30.39it/s]


TCGA-GBM


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 858/858 [00:28<00:00, 29.62it/s]


TCGA-HNSC


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 472/472 [00:16<00:00, 29.06it/s]


In [35]:
WSI_mapping = pd.DataFrame([(k, v) for k, v in wsi_mapping.items()], columns=('submitter_id', 'WSI'))
clinical_tcga = clinical_tcga.merge(WSI_mapping, how='left', on='submitter_id')
clinical_tcga.shape

(11428, 29)

In [36]:
# keep only patients at least with a WSI
clinical_tcga = clinical_tcga[clinical_tcga.WSI.notna()]

In [37]:
pretrain_idx, fine_tune_idx = train_test_split(
    clinical_tcga,
    test_size=0.4,
    random_state=42,
    stratify=clinical_tcga['project_id'] 
)

# Then split train+val into train/val
pretrain_idx, val_pretrain_idx = train_test_split(
    pretrain_idx,
    test_size=0.05,  
    random_state=42
)

def assign_split(submitter_id):
    if submitter_id in pretrain_idx['submitter_id'].values:
        return "pretrain"
    elif submitter_id in val_pretrain_idx['submitter_id'].values:
        return "val-pretrain"
    elif submitter_id in fine_tune_idx['submitter_id'].values:
        return "fine-tune"
    else:
        return pd.NA  

clinical_tcga["split"] = clinical_tcga["submitter_id"].apply(assign_split)

# 4. RNA-Seq <a name="p4"></a>
For RNA Seq we use the snippet code from [MMP](https://github.com/mahmoodlab/MMP/blob/main/src/preprocess_pancancer_TCGA_normalized_RNA.ipynb) to download PAN-Cancer RNASeq for each project

In [43]:
projects = """
TCGA-ACC   TCGA-BRCA     TCGA-CESC  TCGA-COAD  TCGA-ESCA  TCGA-HNSC  TCGA-KIRC  TCGA-LGG   TCGA-LUAD  TCGA-MESO  TCGA-PAAD  TCGA-PRAD  TCGA-SARC  TCGA-STAD  TCGA-THCA  TCGA-UCEC  TCGA-UVM
TCGA-BLCA  TCGA-CHOL  TCGA-DLBC  TCGA-GBM   TCGA-KICH  TCGA-KIRP  TCGA-LIHC  TCGA-LUSC  TCGA-OV    TCGA-PCPG  TCGA-READ  TCGA-SKCM  TCGA-TGCT  TCGA-THYM  TCGA-UCS
"""
project_list = projects.split()
os.makedirs("./data_rna", exist_ok=True)

In [44]:
project_list = [p.replace('-', '.') for p in project_list]
for project in project_list:
    url = f"https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/{project}.sampleMap%2FHiSeqV2_PANCAN.gz"
    out_gz = f"./data_rna/HiSeqV2_PANCAN_{project}.gz"
    out_txt = out_gz.replace(".gz", "")

    print(f"Downloading {project}...")
    os.system(f"curl -s -o {out_gz} '{url}'")

    if os.path.exists(out_gz):
        print(f"Unzipping {project}...")
        os.system(f"gunzip -f {out_gz}")
    else:
        print(f"Failed to download {project}.")

Unzipping TCGA.ACC...
Unzipping TCGA.BRCA...
Unzipping TCGA.CESC...
Unzipping TCGA.COAD...
Unzipping TCGA.ESCA...
Unzipping TCGA.HNSC...
Unzipping TCGA.KIRC...
Unzipping TCGA.LGG...
Unzipping TCGA.LUAD...
Unzipping TCGA.MESO...
Unzipping TCGA.PAAD...
Unzipping TCGA.PRAD...
Unzipping TCGA.SARC...
Unzipping TCGA.STAD...
Unzipping TCGA.THCA...
Unzipping TCGA.UCEC...
Unzipping TCGA.UVM...
Unzipping TCGA.BLCA...
Unzipping TCGA.CHOL...
Unzipping TCGA.DLBC...
Unzipping TCGA.GBM...
Unzipping TCGA.KICH...
Unzipping TCGA.KIRP...
Unzipping TCGA.LIHC...
Unzipping TCGA.LUSC...
Unzipping TCGA.OV...
Unzipping TCGA.PCPG...
Unzipping TCGA.READ...
Unzipping TCGA.SKCM...
Unzipping TCGA.TGCT...
Unzipping TCGA.THYM...
Unzipping TCGA.UCS...


In [45]:
rna_dataframe = []
for subproject in os.listdir('data_rna'):
    temp_df = pd.read_csv(os.path.join('data_rna', subproject), sep='\t').T
    temp_df.columns = temp_df.iloc[0]
    rna_dataframe.append(temp_df[1:])

In [46]:
rna_dataframe = pd.concat(rna_dataframe)

In [47]:
rna_dataframe['patient_id'] = rna_dataframe.index
rna_dataframe['patient_id'] = rna_dataframe['patient_id'].apply(lambda x: str(x)[:-3])
rna_dataframe['is_01'] = rna_dataframe.index.str.endswith('-01')

In [48]:
rna_dataframe = rna_dataframe.sort_values(by=['patient_id', 'is_01'], ascending=[True, False])

In [49]:
rna_dataframe = rna_dataframe.drop_duplicates(subset='patient_id', keep='first')

In [50]:
rna_dataframe = rna_dataframe.set_index('patient_id')
rna_dataframe.drop(columns=['is_01'], inplace=True)

In [51]:
rna_present = pd.DataFrame({
    'submitter_id': rna_dataframe.index,
    'rna': ['yes'] * len(rna_dataframe)
})

In [52]:
clinical_tcga = clinical_tcga.merge(rna_present, how='left', on='submitter_id')
clinical_tcga

,submitter_id,project_id,gender,race,age_at_diagnosis,prior_malignancy,prior_treatment,synchronous_malignancy,tumor_stage,treatments_pharmaceutical_treatment_or_therapy,...,censorship,age,site,oncotree_code,survival_months_dss,censorship_dss,response,WSI,split,rna
0,TCGA-DD-AAVP,TCGA-LIHC,male,asian,17833,no,No,No,stage i,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-LIHC/TCGA-DD-AAVP-0...,pretrain,yes
1,TCGA-KK-A7B2,TCGA-PRAD,male,white,24845,no,No,No,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-PRAD/TCGA-KK-A7B2-0...,fine-tune,yes
2,TCGA-DC-6158,TCGA-READ,male,white,25842,no,No,No,stage i,no,...,0.0,70.0,DC,READ,11.133333,1.0,NaN,[UNI2-h_features/TCGA/TCGA-READ/TCGA-DC-6158-0...,pretrain,yes
3,TCGA-DD-A4NP,TCGA-LIHC,male,white,13124,NaN,Yes,NaN,stage i,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-LIHC/TCGA-DD-A4NP-0...,fine-tune,yes
4,TCGA-HQ-A5ND,TCGA-BLCA,male,not reported,28639,NaN,Yes,NaN,stage iv,no,...,0.0,78.0,HQ,BLCA,9.133333,0.0,NaN,[UNI2-h_features/TCGA/TCGA-BLCA/TCGA-HQ-A5ND-0...,pretrain,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9542,TCGA-XS-A8TJ,TCGA-CESC,female,black or african american,15298,no,No,No,NaN,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-CESC/TCGA-XS-A8TJ-0...,pretrain,yes
9543,TCGA-BP-4790,TCGA-KIRC,male,white,28076,no,No,No,stage i,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-KIRC/TCGA-BP-4790-0...,pretrain,yes
9544,TCGA-N9-A4Q4,TCGA-UCS,female,black or african american,22575,no,No,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-UCS/TCGA-N9-A4Q4-01...,fine-tune,yes
9545,TCGA-RY-A847,TCGA-LGG,male,white,16732,no,No,No,NaN,no,...,1.0,45.0,RY,ODG,NaN,NaN,NaN,[UNI2-h_features/TCGA/TCGA-LGG/TCGA-RY-A847-01...,pretrain,yes


In [53]:
rna_dataframe = rna_dataframe[rna_dataframe.index.isin(clinical_tcga.submitter_id)]

# Keep only based on RNA pathways

In [54]:
hallmarks = pd.read_csv('pathways/hallmarks_comps.csv').rename(columns={'Unnamed: 0': 'gene'})
hallmarks_intersection = list(set(hallmarks.iloc[:, 0]) & set(rna_dataframe.columns))

In [55]:
rna_dataframe_hallmarks = rna_dataframe[hallmarks_intersection]

### Save pathways mappings

In [57]:
hallmarks_genes = {}
for column in hallmarks.columns:
    if column == 'gene':
        continue
    genes = hallmarks.gene[hallmarks[column] == 1].tolist()
    genes = [gene for gene in genes if gene in list(hallmarks_intersection)]
    hallmarks_genes[column] = genes

In [60]:
with open('mappings/hallmarks_genes.json', 'w') as f:
    # write the dictionary to the file in JSON format
    json.dump(hallmarks_genes, f)

# 5. Copy Number Variations (CNV) <a name="p5"></a>

In [151]:
%%time
CNV_files = request_file_info(data_type='Genotyping Array')
CNV_files.shape

CPU times: user 391 ms, sys: 69 ms, total: 460 ms
Wall time: 1min 12s


(147734, 7)

In [152]:
CNV_files = CNV_files[CNV_files['cases.0.project.project_id'].str.startswith('TCGA')]
CNV_files = CNV_files[CNV_files['file_name'].str.endswith('gene_level_copy_number.v36.tsv')]
CNV_files = CNV_files[~CNV_files["file_name"].str.contains("absolute_liftover", na=False)]
CNV_files = CNV_files[~CNV_files["file_name"].str.contains("ascat3", na=False)]
CNV_files.shape

(11104, 7)

In [153]:
print('All rows:       ', CNV_files.shape[0])
print('Unique patients:', CNV_files['cases.0.submitter_id'].unique().shape[0])

All rows:        11104
Unique patients: 10763


In [154]:
# select primary if possible
CNV_files = CNV_files.groupby('cases.0.submitter_id', group_keys=False).apply(select_primary).reset_index(drop=True)

/tmp/ipykernel_3256672/1562612416.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  CNV_files = CNV_files.groupby('cases.0.submitter_id', group_keys=False).apply(select_primary).reset_index(drop=True)


In [155]:
print('All rows:       ', CNV_files.shape[0])
print('Unique patients:', CNV_files['cases.0.submitter_id'].unique().shape[0])

All rows:        10763
Unique patients: 10763


In [156]:
base_path_cnv = '/path/to/your/cnv/'
cnv_mapping = make_patient_file_map(CNV_files, base_path_cnv)

In [157]:
# get protein coding genes
sample = pd.read_csv('./rna_sample/66e79734-9053-4f61-9a32-bee376f5dab0/a4a57e8f-cf4f-4322-b7fe-2336a58cd50f.rna_seq.augmented_star_gene_counts.tsv', sep='\t',skiprows=6, names=['gene_id', 'gene_name','gene_type', 'unstranded', 'stranded_first', 'stranded_second', 	'tpm_unstranded', 'fpkm_unstranded', 'fpkm_uq_unstranded'])
protein_coding_genes = sample[sample.gene_type == 'protein_coding'].gene_name.values.tolist()


In [159]:
def drop_duplicates_prioritize_non_nan(df, column="gene_name"):
    """
    Drops duplicates in a specified column, prioritizing non-NaN values.
    If all duplicates are NaN, keeps only one.
    Preserves the original order.
    """
    # Identify duplicates
    mask = df.duplicated(subset=[column], keep=False)

    # Separate duplicated and unique rows
    duplicated_rows = df[mask]
    unique_rows = df[~mask]

    # Within duplicates, keep first non-NaN value, otherwise first occurrence
    cleaned_duplicates = (
        duplicated_rows.sort_values(by=column, key=lambda x: x.notna(), ascending=False)
        .drop_duplicates(subset=[column], keep="first")
    )

    # Concatenate unique rows with cleaned duplicates and restore order
    cleaned_df = pd.concat([unique_rows, cleaned_duplicates]).sort_index()

    return cleaned_df
    
# keep only protein coding gene
output_path = 'path/to/put/preprocessed/omics_data/TCGA/CNV/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

for patient, filepath in tqdm.tqdm(cnv_mapping.items()):
    idx, filename = filepath.split('/')[-2:]
    if not os.path.exists(os.path.join(output_path, idx)):
        os.makedirs(os.path.join(output_path, idx))

    temp_df = pd.read_csv(filepath, sep='\t')[['gene_id', 'gene_name', 'copy_number']]
    temp_df = temp_df[temp_df.gene_name.isin(protein_coding_genes)]
    temp_df = drop_duplicates_prioritize_non_nan(temp_df)
    temp_df.to_csv(os.path.join(output_path, idx, filename.replace('tsv', 'csv')))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10761/10761 [13:15<00:00, 13.52it/s]


In [160]:
# upate cnv_mapping
cnv_mapping = {k: v.replace(base_path_cnv, output_path).replace('tsv', 'csv') for k, v in cnv_mapping.items()}

In [166]:
CNV_mapping = pd.DataFrame([(k, 'yes') for k in cnv_mapping.keys()], columns=('submitter_id', 'cnv'))
clinical_tcga = clinical_tcga.merge(CNV_mapping, how='left', on='submitter_id')

In [303]:
CNV_mapping_path = pd.DataFrame([(k, v) for k, v in cnv_mapping.items()], columns=('submitter_id', 'cnv_path'))
clinical_tcga = clinical_tcga.merge(CNV_mapping_path, how='left', on='submitter_id')

Now, to group genes together, we propose to do:
1. Keep only those with the greatest variance and group them by chromosome (and by position on the chromosome).

### 1.

In [310]:
subset_pretrain = clinical_tcga[clinical_tcga.split == 'pretrain']
full_cnv = pd.concat([pd.read_csv(value, index_col=2).copy_number.rename(key) for key, value in zip(clinical_tcga[clinical_tcga.cnv_path.notna()].submitter_id, clinical_tcga[clinical_tcga.cnv_path.notna()].cnv_path)], axis=1)
pretrain_cnv = pd.concat([pd.read_csv(value, index_col=2).copy_number.rename(key) for key, value in zip(subset_pretrain[subset_pretrain.cnv_path.notna()].submitter_id, subset_pretrain[subset_pretrain.cnv_path.notna()].cnv_path)], axis=1)

In [311]:
pretrain_cnv.shape, full_cnv.shape

((19925, 5254), (19925, 9210))

In [312]:
pretrain_cnv = pretrain_cnv.T.dropna(axis=1, how="all")
print(pretrain_cnv.shape)
pretrain_cnv = pretrain_cnv.loc[:, ~pretrain_cnv.columns.duplicated(keep="first")]
print(pretrain_cnv.shape)
pretrain_cnv = pretrain_cnv.dropna(thresh=pretrain_cnv.shape[0] * 0.99, axis=1)
print(pretrain_cnv.shape)

(5254, 19890)
(5254, 19890)
(5254, 19868)


In [316]:
sample = pd.read_csv('samples/TCGA-LUAD.b9aad986-ad82-4366-b51e-8895eff15463.gene_level_copy_number.v36.csv')
keep = sample[~sample.chromosome.isin(['chrX', 'chrY'])].gene_name.values.tolist()
pretrain_cnv = pretrain_cnv[list(set(pretrain_cnv.columns) & set(keep))]

In [317]:
pretrain_cnv.shape

(5254, 18979)

In [318]:
std_dev = pretrain_cnv.std()
std_dev = std_dev.nlargest(6750)

In [319]:
filtered_genes = std_dev.index
len(filtered_genes)

6750

In [320]:
sample = sample[sample.gene_name.isin(filtered_genes)]

In [276]:
chromosomes_cluster = {
    'chr1': 7,
    'chr7': 6,
    'chr3': 4,
    'chr4;chr5': 1,
    'chr17': 4,
    'chr8': 4,
    'chr20': 4,
    'chr12': 3,
    'chr2;chr22;chr21': 1,
    'chr14;chr15;chr16': 1,
    'chr19': 3,
    'chr6': 2,
    'chr11': 2,
    'chr13': 1,
    'chr9;chr10': 1,
    'chr18': 1,
}    

In [321]:
cnv_chr_genes = {}
for chrom_group, clusters in chromosomes_cluster.items():
    chrom_list = chrom_group.split(';') 
    chrom_data = sample[sample["chromosome"].isin(chrom_list)]
    if clusters == 1:
        cnv_chr_genes[chrom_group] = chrom_data.gene_name.values.tolist()
    else:
        chrom_data.sort_values(by="start", inplace=True)
        # Assign clusters evenly based on the number of genes
        chrom_data["cluster"] = np.repeat(range(1, clusters + 1), np.ceil(len(chrom_data) / clusters))[:len(chrom_data)]
        
        for i in range(clusters):
            cnv_chr_genes[f'{chrom_group}-{i+1}'] = chrom_data[chrom_data.cluster == i+1].gene_name.values.tolist()

/tmp/ipykernel_3256672/3314043594.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chrom_data.sort_values(by="start", inplace=True)
/tmp/ipykernel_3256672/3314043594.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chrom_data["cluster"] = np.repeat(range(1, clusters + 1), np.ceil(len(chrom_data) / clusters))[:len(chrom_data)]
/tmp/ipykernel_3256672/3314043594.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [322]:
with open('mappings/cnv_chr_genes.json', 'w') as f:
    # write the dictionary to the file in JSON format
    json.dump(cnv_chr_genes, f)

In [323]:
full_cnv = full_cnv.T[filtered_genes]

# 6. DNAm <a name="p6"></a>

In [194]:
%%time 
DNAm_files = request_file_info(data_type='Methylation Array')
DNAm_files.shape

CPU times: user 138 ms, sys: 38.8 ms, total: 176 ms
Wall time: 22.6 s


(50913, 10)

In [196]:
DNAm_files = DNAm_files[DNAm_files['cases.0.project.project_id'].str.startswith('TCGA')]
DNAm_files = DNAm_files[DNAm_files['file_name'].str.endswith('level3betas.txt')]
DNAm_files.shape

(12527, 10)

In [197]:
print('All rows:       ', DNAm_files.shape[0])
print('Unique patients:', DNAm_files['cases.0.submitter_id'].unique().shape[0])

All rows:        12527
Unique patients: 11000


In [198]:
DNAm_files = DNAm_files.groupby('cases.0.submitter_id', group_keys=False).apply(select_primary).reset_index(drop=True)

/tmp/ipykernel_3256672/2877964513.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  DNAm_files = DNAm_files.groupby('cases.0.submitter_id', group_keys=False).apply(select_primary).reset_index(drop=True)


In [199]:
print('All rows:       ', DNAm_files.shape[0])
print('Unique patients:', DNAm_files['cases.0.submitter_id'].unique().shape[0])

All rows:        11000
Unique patients: 11000


In [201]:
base_path_raw_dnam = 'path/to/data/from/gdc'
sample_480k = pd.read_csv(os.path.join(base_path_raw_dnam, 'f366f277-4079-449d-a781-0a0492a3ae41', 'e3976274-ad65-4d85-bf5c-9c10f2502748.methylation_array.sesame.level3betas.txt'), sep='\t', index_col=0, header=None)
sample_27k = pd.read_csv(os.path.join(base_path_raw_dnam, '61fdc0ee-31d6-4051-bc1c-64feaa2e4042', 'fbc63a44-89c8-411c-87c9-05fd4138647a.methylation_array.sesame.level3betas.txt'), sep='\t', index_col=0, header=None)
sample_850k = pd.read_csv(os.path.join('base_path_raw_dnam' '/cptac_wsi/090ad67e-980a-4e8a-9e8b-a231108d871c/8ec2e06d-d893-4af0-9d7d-2796d34c2374.methylation_array.sesame.level3betas.txt', sep='\t', index_col=0, header=None))

In [202]:
probe_set = [p for p in list(sample_27k.index) if p in sample_480k.index]
probe_set = [p for p in probe_set if p in sample_850k.index]
len(probe_set)

24655

In [203]:
# keep only overlapping probes
output_path = '/path/for/processed/dnam'

if not os.path.exists(output_path):
    os.makedirs(output_path)

for idx in tqdm.tqdm(os.listdir(base_path_raw_dnam)):
    if not os.path.exists(os.path.join(output_path, idx)):
        os.makedirs(os.path.join(output_path, idx))
    
    files = os.listdir(os.path.join(base_path_raw_dnam, idx))
    filename = [file for file in files if 'level3betas.txt' in file][0]
    temp_df = pd.read_csv(os.path.join(base_path_raw_dnam, idx, filename), sep='\t', index_col=0, header=None, names=['Composite Element REF', 'Beta_value'])
    temp_df = temp_df[temp_df.index.isin(probe_set)]
    temp_df.to_csv(os.path.join(output_path, idx, filename.replace('txt', 'csv')))

 71%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                              | 8840/12527 [1:16:31<38:00,  1.62it/s]/tmp/ipykernel_3256672/1359710148.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(input_path, idx, filename), sep='\t', index_col=0, header=None, names=['Composite Element REF', 'Beta_value'])
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12527/12527 [1:54:18<00:00,  1.83it/s]


In [204]:
file_map = make_patient_file_map(DNAm_files, base_dir=base_path_raw_dnam)
file_map = {k: file_map[k] for k in file_map if k in list(clinical_tcga['submitter_id'])}
len(file_map)

9333

In [205]:
file_map = {k:v.replace(base_path_raw_dnam, output_path).replace('.txt', '.csv') for k,v in file_map.items()}

In [206]:
DNAm_mapping = pd.DataFrame([(k, 'yes') for k in file_map.keys()], columns=('submitter_id', 'dnam'))
clinical_tcga = clinical_tcga.merge(DNAm_mapping, how='left', on='submitter_id')

In [281]:
DNAm_mapping_path = pd.DataFrame([(k, v) for k, v in file_map.items()], columns=('submitter_id', 'dnam_path'))
clinical_tcga = clinical_tcga.merge(DNAm_mapping_path, how='left', on='submitter_id')

In [282]:
subset_pretrain = clinical_tcga[clinical_tcga.split == 'pretrain']
pretrain_dnam = pd.concat([pd.read_csv(value, index_col=0).Beta_value.rename(key) for key, value in zip(subset_pretrain[subset_pretrain.dnam_path.notna()].submitter_id, subset_pretrain[subset_pretrain.dnam_path.notna()].dnam_path)], axis=1)
full_dnam = pd.concat([pd.read_csv(value, index_col=0).Beta_value.rename(key) for key, value in zip(clinical_tcga[clinical_tcga.dnam_path.notna()].submitter_id, clinical_tcga[clinical_tcga.dnam_path.notna()].dnam_path)], axis=1)

In [283]:
full_dnam = full_dnam.T
pretrain_dnam = pretrain_dnam.T
full_dnam.shape, pretrain_dnam.shape

((9333, 24655), (5321, 24655))

In [288]:
pretrain_dnam = pretrain_dnam.dropna(axis=1, how="all")
print(pretrain_dnam.shape)
pretrain_dnam = pretrain_dnam.loc[:, ~pretrain_dnam.columns.duplicated(keep="first")]
print(pretrain_dnam.shape)
pretrain_dnam = pretrain_dnam.dropna(thresh=pretrain_dnam.shape[0] * 0.99, axis=1)
print(pretrain_dnam.shape)

(5321, 23214)
(5321, 23214)
(5321, 20600)


In [289]:
pretrain_dnam.shape

(5321, 20600)

In [290]:
infinium = pd.read_csv('mappings/HM450.hg38.manifest.gencode.v36.tsv', sep='\t')
infinium = infinium[infinium.probeID.isin(pretrain_dnam.columns)]
keep = infinium[~infinium.CpG_chrm.isin(['chrX', 'chrY'])].probeID.values.tolist()
infinium = infinium[infinium.probeID.isin(keep)]
pretrain_dnam = pretrain_dnam[keep]

In [291]:
std_dev = pretrain_dnam.std()
std_dev = std_dev.nlargest(7150)

In [292]:
filtered_probes = std_dev.index
len(filtered_probes)

7150

In [293]:
full_dnam = full_dnam[filtered_probes]
pretrain_dnam = pretrain_dnam[filtered_probes]

In [294]:
infinium_filtered = infinium[infinium.probeID.isin(filtered_probes)]

In [295]:
dnam_chromosome_cluster = {
    'chr1': 6,
    'chr19': 4,
    'chr11': 4,
    'chr17': 3,
    'chr2': 3,
    'chr3': 3,
    'chr6': 3,
    'chr12': 3,
    'chr7': 2,
    'chr5': 2,
    'chr8': 2,
    'chr20': 2,
    'chr16': 2,
    'chr10': 2,
    'chr4': 2,
    'chr9': 2,
    'chr15': 1,
    'chr14': 1,
    'chr22': 1,
    'chr13': 1,
    'chr18': 1,
    'chr21': 1
}

In [296]:
dnam_genes = {}
for chrom, clusters in dnam_chromosome_cluster.items():
    chrom_data = infinium_filtered[infinium_filtered["CpG_chrm"] == chrom]
    if clusters == 1:
        dnam_genes[chrom] = chrom_data.probeID.values.tolist()
    else:
        chrom_data.sort_values(by="CpG_beg", inplace=True)
        # Assign clusters evenly based on the number of genes
        chrom_data["cluster"] = np.repeat(range(1, clusters + 1), np.ceil(len(chrom_data) / clusters))[:len(chrom_data)]
        for i in range(clusters):
            dnam_genes[f'{chrom}-{i+1}'] = chrom_data[chrom_data.cluster == i+1].probeID.values.tolist()

/tmp/ipykernel_3256672/2756484623.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chrom_data.sort_values(by="CpG_beg", inplace=True)
/tmp/ipykernel_3256672/2756484623.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chrom_data["cluster"] = np.repeat(range(1, clusters + 1), np.ceil(len(chrom_data) / clusters))[:len(chrom_data)]
/tmp/ipykernel_3256672/2756484623.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [297]:
with open('mappings/dnam_chr_genes.json', 'w') as f:
    # write the dictionary to the file in JSON format
    json.dump(dnam_genes, f)

In [325]:
full_dnam.shape, pretrain_dnam.shape

((9333, 7150), (5321, 7150))

# Split the fine-tune dataset

In [234]:
fine_tune_subset = clinical_tcga[clinical_tcga.split == 'fine-tune']

In [18]:
def get_folding(metadata, finetune_df):
    skf = KFold(n_splits=5, shuffle=True, random_state=1999)
    folds = {}  # Dictionary to store new folds
    for fold, (_, val_idx) in enumerate(skf.split(finetune_df['submitter_id'])):
        print(fold)
        for idx in val_idx:
            x_value = finetune_df.iloc[idx]['submitter_id']
            folds[x_value] = fold

    folding_df = pd.DataFrame([(k, v) for k, v in folds.items()], columns=('submitter_id', 'folds'))
    metadata = metadata.merge(folding_df, how='left', on='submitter_id')
    return metadata

In [327]:
clinical_tcga.drop(columns=['dnam_path', 'cnv_path'], inplace=True)

# Divise in subfolder for fine-tuning task

In [60]:
substudies = ['gbmlgg', 'luad', 'brca', 'coadread']
def make_data(inp_project):
    if inp_project == 'gbmlgg':
        project = ['TCGA-GBM', 'TCGA-LGG']
    elif inp_project == 'coadread':
        project = ['TCGA-COAD', 'TCGA-READ']
    else:
        project = [f'TCGA-{inp_project.upper()}']

    print(f"Processing input project: {inp_project}")
    subset = clinical_tcga[clinical_tcga.project_id.isin(project)]
    subset_finetune = subset[subset.split == 'fine-tune']
    subset_finetune = subset_finetune.dropna(subset=['rna'])
    subset_finetune = subset_finetune[subset_finetune.survival_months.notna()]
    subset = get_folding(subset, subset_finetune)
    #subset = subset[subset.survival_months.notna()]
    #subset = subset[subset.split == 'fine-tune']
    print(subset.shape)
    print(subset.folds.value_counts())
    print("Handling RNA data")
    hallmark = rna_dataframe_hallmarks[rna_dataframe_hallmarks.index.isin(subset.submitter_id.tolist())]
    print('RNA shape: ', hallmark.shape)
    hallmark.to_csv(f'../datasets_csv/rna/hallmarks/tcga_{inp_project}.csv')
    print("RNA.... done")
    print("Handling DNAm")
    dnam = full_dnam[full_dnam.index.isin(subset.submitter_id.tolist())]
    print("DNAm shape: ", dnam.shape)
    dnam.to_csv(f'../datasets_csv/dnam/tcga_{inp_project}.csv')
    print("DNAm.... done")
    print("Handling CNV")
    cnv = full_cnv[full_cnv.index.isin(subset.submitter_id.tolist())]
    print("CNV shape: ", cnv.shape)
    cnv.to_csv(f'../datasets_csv/cnv/tcga_{inp_project}.csv')

    print("SAVING METADATA")
    subset.to_csv(f'../datasets_csv/metadata/tcga_{inp_project}.csv')
    print(subset.shape)

for s in substudies:
    make_data(s)

Processing input project: kirc
0
1
2
3
4
(513, 35)
folds
1.0    28
0.0    28
3.0    27
2.0    27
4.0    27
Name: count, dtype: int64
Handling RNA data
RNA shape:  (510, 4168)
RNA.... done
Handling DNAm
DNAm shape:  (511, 7150)
DNAm.... done
Handling CNV
CNV shape:  (497, 6750)
SAVING METADATA
(513, 35)


## Save pan-cancer data

In [339]:
clinical_tcga.to_csv('../datasets_csv/metadata/tcga_pancan.csv')
rna_dataframe_hallmarks.to_csv('../datasets_csv/rna/hallmarks/tcga_pancan.csv')
full_dnam.to_csv('../datasets_csv/dnam/tcga_pancan.csv')
full_cnv.to_csv('../datasets_csv/cnv/tcga_pancan.csv')

In [16]:
clinical_tcga.to_csv('../datasets_csv/metadata/tcga_pancan.csv')